In [1]:
%autoawait
import time
import asyncio
import pprint
from termcolor import colored,cprint
from aries_basic_controller.aries_controller import AriesAgentController

# Create a small utility to print json formatted outout more human-readable    
pp = pprint.PrettyPrinter(indent=4)

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://bob-agent:8021"

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, mediation=True)


## Listen for webhooks

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}
agent_controller.register_listeners([cred_listener, connection_listener], defaults=True)

In [4]:
### Time to create an invitation

In [5]:
# Create Invitation
unmediated_invite = await agent_controller.connections.create_invitation()

Connection message {'their_role': 'invitee', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:01.161870Z', 'accept': 'auto', 'rfc23_state': 'invitation-sent', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:01.161870Z', 'invitation_key': 'C2u93XSU1xttbZCejAXMeaJkiHM24Gs7uBQkqYdYcJV4', 'connection_id': '07cd6078-1a0a-4d6b-965f-21f77552c1c7'} 07cd6078-1a0a-4d6b-965f-21f77552c1c7
Connection message {'their_role': 'invitee', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:01.161870Z', 'accept': 'auto', 'rfc23_state': 'invitation-sent', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:01.161870Z', 'invitation_key': 'C2u93XSU1xttbZCejAXMeaJkiHM24Gs7uBQkqYdYcJV4', 'connection_id': '07cd6078-1a0a-4d6b-965f-21f77552c1c7'} 07cd6078-1a0a-4d6b-965f-21f77552c1c7


Going to the mediation agent1 to generate and fetch the invitation

In [7]:
mediator_invitation ={'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'bd674b32-b564-4d4d-ac58-2eb2e5c54b84', 'recipientKeys': ['4tX7b9jKXcMWUtP2B7BjssDYAibB8QhuYPwQ1uwr5g5S'], 'label': 'MEDIATOR1', 'serviceEndpoint': 'https://b211401e2898.ngrok.io'}

In [8]:
response = await agent_controller.connections.accept_connection(mediator_invitation)
pp.pprint(response)

Connection message {'their_role': 'inviter', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:25.799071Z', 'their_label': 'MEDIATOR1', 'accept': 'manual', 'rfc23_state': 'invitation-received', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:25.799071Z', 'invitation_key': '4tX7b9jKXcMWUtP2B7BjssDYAibB8QhuYPwQ1uwr5g5S', 'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d'} 31867543-7afe-4d69-9515-b2f6add2616d
Connection message {'their_role': 'inviter', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:25.799071Z', 'their_label': 'MEDIATOR1', 'accept': 'manual', 'rfc23_state': 'invitation-received', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:25.799071Z', 'invitation_key': '4tX7b9jKXcMWUtP2B7BjssDYAibB8QhuYPwQ1uwr5g5S', 'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d'} 31867543-7afe-4d69-9515-b2f6add2616d
Connection message {'my_did': 'W8GY23CjwmZnxJ2f1wxJFF', 'their_role': 'inviter', 'state'

Connection message {'my_did': 'W8GY23CjwmZnxJ2f1wxJFF', 'their_role': 'inviter', 'state': 'active', 'their_did': 'PArPaWr5dkTx4Ad87z8mAj', 'request_id': '2d711d87-48d3-485f-ace7-b2bd1f0ab864', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:25.799071Z', 'their_label': 'MEDIATOR1', 'accept': 'manual', 'rfc23_state': 'completed', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:27.164371Z', 'invitation_key': '4tX7b9jKXcMWUtP2B7BjssDYAibB8QhuYPwQ1uwr5g5S', 'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d'} 31867543-7afe-4d69-9515-b2f6add2616d
Connection message {'my_did': 'W8GY23CjwmZnxJ2f1wxJFF', 'their_role': 'inviter', 'state': 'active', 'their_did': 'PArPaWr5dkTx4Ad87z8mAj', 'request_id': '2d711d87-48d3-485f-ace7-b2bd1f0ab864', 'routing_state': 'none', 'created_at': '2021-04-02 11:58:25.799071Z', 'their_label': 'MEDIATOR1', 'accept': 'manual', 'rfc23_state': 'completed', 'invitation_mode': 'once', 'updated_at': '2021-04-02 11:58:27.164371Z', 'invitation_key': 

In [9]:
connection_id = response["connection_id"]
print(connection_id)

31867543-7afe-4d69-9515-b2f6add2616d



## Now we will Request mediation

Now that we have successfully established a connection between Bob and the mediator agent1 we can proceed to request mediation from the mediator1.


In [11]:
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)
print(connection['state'])

active


In [12]:
# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Bob's subwallet agent")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    print(mediation_req)


{'role': 'client', 'state': 'request', 'mediation_id': '7b01ae71-f559-4703-8cf9-5a5abde11566', 'created_at': '2021-04-02 11:58:54.521220Z', 'mediator_terms': [], 'updated_at': '2021-04-02 11:58:54.521220Z', 'recipient_terms': [], 'routing_keys': [], 'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d'}


Now we have to take a look at the mediation records and we should see our mediation in there

In [13]:
response = await agent_controller.mediation.get_mediation_records()
print(response)

[{'role': 'client', 'state': 'granted', 'mediation_id': '7b01ae71-f559-4703-8cf9-5a5abde11566', 'created_at': '2021-04-02 11:58:54.521220Z', 'mediator_terms': [], 'updated_at': '2021-04-02 11:58:55.338129Z', 'endpoint': 'https://b211401e2898.ngrok.io', 'recipient_terms': [], 'routing_keys': ['AfHREDp3d5SPiT9Pe7neDWjg7MgvfbdGRwSkVQoDQ7CP'], 'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d'}]



Here we are Setting a default mediator

By setting a default mediator, all DIDComm connections we establish will be routed through this mediator. To do this we use the appropriate API endpoint via the agent controller and pass it the mediation ID of our mediated connection.


In [14]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
pp.pprint(default_mediation_res)

{   'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d',
    'created_at': '2021-04-02 11:58:54.521220Z',
    'endpoint': 'https://b211401e2898.ngrok.io',
    'mediation_id': '7b01ae71-f559-4703-8cf9-5a5abde11566',
    'mediator_terms': [],
    'recipient_terms': [],
    'role': 'client',
    'routing_keys': ['AfHREDp3d5SPiT9Pe7neDWjg7MgvfbdGRwSkVQoDQ7CP'],
    'state': 'granted',
    'updated_at': '2021-04-02 11:58:55.338129Z'}


## Check to see our default mediator actully there

In [15]:
default_mediator = await agent_controller.mediation.get_default_mediator()
pp.pprint(default_mediator)

if default_mediator['connection_id'] != connection_id:
    print("Oooops! Something went wrong setting the default mediator. Please, check above and try again")
else:
    print("\n\n Hooray! We have succesfully set a default mediator.")


{   'connection_id': '31867543-7afe-4d69-9515-b2f6add2616d',
    'created_at': '2021-04-02 11:58:54.521220Z',
    'endpoint': 'https://b211401e2898.ngrok.io',
    'mediation_id': '7b01ae71-f559-4703-8cf9-5a5abde11566',
    'mediator_terms': [],
    'recipient_terms': [],
    'role': 'client',
    'routing_keys': ['AfHREDp3d5SPiT9Pe7neDWjg7MgvfbdGRwSkVQoDQ7CP'],
    'state': 'granted',
    'updated_at': '2021-04-02 11:58:55.338129Z'}


 Hooray! We have succesfully set a default mediator.


In [20]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)
pp.pprint("Routing Key : " + invite_message['routingKeys'][0])

Connection message {'their_role': 'invitee', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 12:00:05.928109Z', 'accept': 'auto', 'rfc23_state': 'invitation-sent', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:00:05.928109Z', 'invitation_key': '56TcAEBtXogPn2kjgAJLyw15FefLx2RcjXdrnyp7gujT', 'connection_id': '81d96ecb-e0bf-4cd9-bce7-782f9e37ef2f'} 81d96ecb-e0bf-4cd9-bce7-782f9e37ef2f
Connection message {'their_role': 'invitee', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 12:00:05.928109Z', 'accept': 'auto', 'rfc23_state': 'invitation-sent', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:00:05.928109Z', 'invitation_key': '56TcAEBtXogPn2kjgAJLyw15FefLx2RcjXdrnyp7gujT', 'connection_id': '81d96ecb-e0bf-4cd9-bce7-782f9e37ef2f'} 81d96ecb-e0bf-4cd9-bce7-782f9e37ef2f
Connection ID 81d96ecb-e0bf-4cd9-bce7-782f9e37ef2f
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '4a61a216-c


## Checking routing keys

This routing key should be used from now on to encrypt all messages/comminucation. Below you'll see that going back our initially created invitation we don't have such key.


In [21]:
unmediated_invite_message = unmediated_invite['invitation']
pp.pprint(unmediated_invite_message)

{   '@id': 'c8e48f58-aeec-4ee3-935b-0326dd990eca',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'BOB',
    'recipientKeys': ['C2u93XSU1xttbZCejAXMeaJkiHM24Gs7uBQkqYdYcJV4'],
    'serviceEndpoint': 'https://a4fa7fab5de2.ngrok.io'}


In [22]:

# Comparing endpoints to check and see the service endpoint
print("Unmediated endpoint: " + unmediated_invite_message['serviceEndpoint'] + "\n\n")
print("Mediated endpoint: " + invite_message['serviceEndpoint'])
print("Mediator Invitation Ednpoint: " + mediator_invitation['serviceEndpoint'])

Unmediated endpoint: https://a4fa7fab5de2.ngrok.io


Mediated endpoint: https://b211401e2898.ngrok.io
Mediator Invitation Ednpoint: https://b211401e2898.ngrok.io


## Now we are connecting with Alice 

In [23]:
alice_invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5a193683-0f1a-4961-b198-fae5d5a70112', 'recipientKeys': ['9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL'], 'serviceEndpoint': 'https://a4e6ac934df2.ngrok.io', 'label': 'ALICE'}

In [24]:
response = await agent_controller.connections.accept_connection(alice_invitation)
pp.pprint(response)

Connection message {'their_role': 'inviter', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 12:01:36.891068Z', 'their_label': 'ALICE', 'accept': 'manual', 'rfc23_state': 'invitation-received', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:01:36.891068Z', 'invitation_key': '9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL', 'connection_id': '253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0'} 253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0
Connection message {'their_role': 'inviter', 'state': 'invitation', 'routing_state': 'none', 'created_at': '2021-04-02 12:01:36.891068Z', 'their_label': 'ALICE', 'accept': 'manual', 'rfc23_state': 'invitation-received', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:01:36.891068Z', 'invitation_key': '9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL', 'connection_id': '253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0'} 253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0
Connection message {'my_did': 'Kpgu44Nnkax9pKoggngk9b', 'their_role': 'inviter', 'state': 'reque

Connection message {'my_did': 'Kpgu44Nnkax9pKoggngk9b', 'their_role': 'inviter', 'state': 'active', 'their_did': '3JAFy7LhTRwJg8c6gmShjL', 'request_id': '034aeb71-76fc-4ab1-8474-fbda229021b8', 'routing_state': 'none', 'created_at': '2021-04-02 12:01:36.891068Z', 'their_label': 'ALICE', 'accept': 'manual', 'rfc23_state': 'completed', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:01:38.818581Z', 'invitation_key': '9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL', 'connection_id': '253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0'} 253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0
Connection message {'my_did': 'Kpgu44Nnkax9pKoggngk9b', 'their_role': 'inviter', 'state': 'active', 'their_did': '3JAFy7LhTRwJg8c6gmShjL', 'request_id': '034aeb71-76fc-4ab1-8474-fbda229021b8', 'routing_state': 'none', 'created_at': '2021-04-02 12:01:36.891068Z', 'their_label': 'ALICE', 'accept': 'manual', 'rfc23_state': 'completed', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:01:38.818581Z', 'invitation_key': '9aZs5Zi

In [25]:
connection_id = response["connection_id"]
print(connection_id)

253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0


In [26]:
## get the connection
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)

{'my_did': 'Kpgu44Nnkax9pKoggngk9b', 'their_role': 'inviter', 'state': 'active', 'their_did': '3JAFy7LhTRwJg8c6gmShjL', 'request_id': '034aeb71-76fc-4ab1-8474-fbda229021b8', 'routing_state': 'none', 'created_at': '2021-04-02 12:01:36.891068Z', 'their_label': 'ALICE', 'accept': 'manual', 'rfc23_state': 'completed', 'invitation_mode': 'once', 'updated_at': '2021-04-02 12:01:38.818581Z', 'invitation_key': '9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL', 'connection_id': '253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0'}


In [27]:
# Let's check for the state
def check_connection(connection):
    if connection['state'] != 'active':
        print("No active connection. \n\nPlease go back and ensure you have established an active connection between the mediator agent and Bob's subwallet agent\n")    
        print("State: " + connection['state']+ "\n")    
    else:
        print("Active connection established\n")
        print("State: " + connection['state']+ "\n")    
        pp.pprint(connection)

check_connection(connection)

Active connection established

State: active

{   'accept': 'manual',
    'connection_id': '253fcd08-fd1f-4ae1-a7b3-2df48d61d7a0',
    'created_at': '2021-04-02 12:01:36.891068Z',
    'invitation_key': '9aZs5Zi56ZtGnrZWD9ByyWVgJvfFKu4EJRPBYf7QECLL',
    'invitation_mode': 'once',
    'my_did': 'Kpgu44Nnkax9pKoggngk9b',
    'request_id': '034aeb71-76fc-4ab1-8474-fbda229021b8',
    'rfc23_state': 'completed',
    'routing_state': 'none',
    'state': 'active',
    'their_did': '3JAFy7LhTRwJg8c6gmShjL',
    'their_label': 'ALICE',
    'their_role': 'inviter',
    'updated_at': '2021-04-02 12:01:38.818581Z'}


In [29]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (external)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

You need to first send a credential from the issuer notebook (external)


# Request Credential from Issuer

In [28]:
try:
    record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
    state = record['state']
    role = record['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)

NameError: name 'ClientResponseError' is not defined

In [ ]:
# Store the credential
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

In [ ]:
try:
    response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
    state = response['state']
    role = response['role']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
except ClientResponseError as err:
    print(err)